In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from dotenv import load_dotenv
import os

app_dir = os.path.join(os.getcwd(), "app")
load_dotenv(os.path.join(app_dir, ".env"))

car_template = """You are an expert in automobiles. You have extensive knowledge about car mechanics, \
models, and automotive technology. You provide clear and helpful answers about cars.

Here is a question:
{query}"""

restaurant_template = """You are a knowledgeable food critic and restaurant reviewer. You have a deep understanding of \
different cuisines, dining experiences, and what makes a great restaurant. You answer questions about restaurants insightfully.

Here is a question:
{query}"""

technology_template = """You are a tech expert with in-depth knowledge of the latest gadgets, software, \
and technological trends. You provide insightful and detailed answers about technology.

Here is a question:
{query}"""

In [2]:
car_questions = [
    "What is the difference between a sedan and an SUV?",
    "How does a hybrid car save fuel?",
    "What should I look for when buying a used car?",
]

restaurant_questions = [
    "What makes a five-star restaurant exceptional?",
    "How do I choose a good wine pairing for my meal?",
    "What are the key elements of French cuisine?",
]

technology_questions = [
    "What are the latest advancements in AI?",
    "How do I secure my home network against cyber threats?",
    "What should I consider when buying a new smartphone?",
]

In [3]:
embeddings = OpenAIEmbeddings()

car_question_embeddings = embeddings.embed_documents(car_questions)
restaurant_question_embeddings = embeddings.embed_documents(restaurant_questions)
technology_question_embeddings = embeddings.embed_documents(technology_questions)

In [4]:
def prompt_router(input):
    query_embedding = embeddings.embed_query(input["query"])
    car_similarity = cosine_similarity([query_embedding], car_question_embeddings)[0]
    restaurant_similarity = cosine_similarity(
        [query_embedding], restaurant_question_embeddings
    )[0]
    technology_similarity = cosine_similarity(
        [query_embedding], technology_question_embeddings
    )[0]

    max_similarity = max(
        max(car_similarity), max(restaurant_similarity), max(technology_similarity)
    )

    if max_similarity == max(car_similarity):
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif max_similarity == max(restaurant_similarity):
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    else:
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)


input_query = {"query": "What's the best way to improve my cars's battery life?"}
prompt = prompt_router(input_query)

Using CAR


In [5]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

In [6]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

Using RESTAURANT


"Identifying a good vintage wine at a restaurant involves a combination of knowledge, intuition, and a bit of research. Here are some key factors to consider:\n\n1. **Know the Basics of Vintage**: Understanding that a wine’s vintage refers to the year in which the grapes were harvested is crucial. Some regions and varietals have exceptional years; for example, Bordeaux from 2010 or Napa Valley Cabernet from 2016. Familiarize yourself with notable vintages for the types of wine you enjoy.\n\n2. **Research the Producer**: A wine's producer can be just as important as its vintage. Some wineries consistently produce high-quality wines, while others may not be as reliable. Look for producers with a strong reputation in their respective regions.\n\n3. **Check the Wine List**: A good restaurant will have a carefully curated wine list. Look for keywords like “reserve,” “grand cru,” or “single vineyard,” which can indicate higher quality. Also, a well-organized wine list — by varietal or region

Classification

In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

classification_template = PromptTemplate.from_template(
    """You are good at classifying a question.
    Given the user question below, classify it as either being about `Car`, `Restaurant`, or `Technology`.

    <If the question is about car mechanics, models, or automotive technology, classify it as 'Car'>
    <If the question is about cuisines, dining experiences, or restaurant services, classify it as 'Restaurant'>
    <If the question is about gadgets, software, or technological trends, classify it as 'Technology'>

    <question>
    {question}
    </question>

    Classification:"""
)

classification_chain = classification_template | ChatOpenAI(model="gpt-4o-mini") | StrOutputParser()

In [9]:
def prompt_router(input):
    classification = classification_chain.invoke({"question": input["query"]})

    if classification == "Car":
        print("Using CAR")
        return PromptTemplate.from_template(car_template)
    elif classification == "Restaurant":
        print("Using RESTAURANT")
        return PromptTemplate.from_template(restaurant_template)
    elif classification == "Technology":
        print("Using TECHNOLOGY")
        return PromptTemplate.from_template(technology_template)
    else:
        print("Unexpected classification:", classification)
        return None


input_query = {"query": "What are the latest trends in electric cars?"}
prompt = prompt_router(input_query)

Using CAR


In [10]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | ChatOpenAI(model="gpt-4o-mini")
    | StrOutputParser()
)

In [11]:
chain.invoke("How do I identify a good vintage wine at a restaurant?")

Using RESTAURANT


"Identifying a good vintage wine at a restaurant involves a combination of knowledge, research, and sensory evaluation. Here are some key points to help you make an informed choice:\n\n1. **Understand the Basics of Vintage**: A vintage wine indicates the year the grapes were harvested. Not all vintages are created equal; factors such as weather conditions greatly influence the quality of the grapes. Research which vintages are considered excellent for the particular wine region or varietal you are interested in.\n\n2. **Know the Regions and Producers**: Familiarize yourself with reputable wine regions (e.g., Bordeaux, Napa Valley, Burgundy) and renowned producers. Wines from celebrated vintages and producers are more likely to be of high quality. \n\n3. **Check the Wine List**: A well-curated wine list can often signal the quality of a restaurant. Look for lists that provide information about the vintages, as well as region and varietal. This can indicate the staff's level of expertise